In [1]:
import os
import sys
import argparse
import cv2
import time
from config_reader import config_reader
from processing import extract_parts, draw

from model.cmu_model import get_testing_model

sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), ".."))

currentDT = time.localtime()
start_datetime = time.strftime("-%m-%d-%H-%M-%S", currentDT)

Using TensorFlow backend.
/home/ashish/Documents/UCD/InsightCentre/VirtualEnvironments/multits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ashish/Documents/UCD/InsightCentre/VirtualEnvironments/multits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ashish/Documents/UCD/InsightCentre/VirtualEnvironments/multits/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood

In [2]:
keras_weights_file = "model.h5"
frame_rate_ratio = 1
process_speed = 4
ending_frame = None

In [ ]:
ffmpeg_extract_subclip("/home/ashish/Downloads/P2 Frontal 2.MP4", 1021.0, 1030, targetname="test.mp4")


In [30]:
# Video input
video = "/home/ashish/Downloads/HumanPose/Binh_VideoDataset_June2019/TimeSeries_Exp4/test.mp4"
video_file = video

# Output location
output_path = 'videos/coordinates/'

In [31]:
# vgg normalization (subtracting mean) on input images
model = get_testing_model()
model.load_weights(keras_weights_file)
params, model_params = config_reader()


In [32]:
# Video reader
cam = cv2.VideoCapture(video_file)
input_fps = cam.get(cv2.CAP_PROP_FPS)
ret_val, orig_image = cam.read()
video_length = int(cam.get(cv2.CAP_PROP_FRAME_COUNT))

if ending_frame is None:
    ending_frame = video_length

In [34]:
xc = []
yc = []
cc = []

In [35]:
process_speed

4

In [36]:
scale_search = [1]  # [.5, 1, 1.5, 2]
scale_search = scale_search[0:process_speed]

params['scale_search'] = scale_search

i = 0  # default is 0
while(cam.isOpened()) and ret_val is True and i < ending_frame:
    if i % frame_rate_ratio == 0:

        input_image = cv2.cvtColor(orig_image, cv2.COLOR_RGB2BGR)

        tic = time.time()

        # generate image with body parts
        body_parts, all_peaks, subset, candidate = extract_parts(input_image, params, model, model_params)

        print('Processing frame: ', i)
        toc = time.time()
        print('processing time is %.5f' % (toc - tic))
        cc.append(body_parts)

    ret_val, orig_image = cam.read()

    i += 1
    
    break

Processing frame:  0
processing time is 5.19527


18

In [10]:
len(cc)

241

In [11]:
import pickle

In [12]:
with open('./cc.pkl', 'wb') as f:
    pickle.dump(cc, f)   # pickle.dump(cc, f)

In [13]:
cc[0]

{'nose': (871, 304),
 'neck': (854, 324),
 'right_shoulder': (839, 320),
 ' right_elbow': (626, 415),
 'right_wrist': (391, 354),
 'left_shoulder': (870, 329),
 'left_elbow': (726, 431),
 'left_wrist': (691, 416),
 'right_hip': (651, 413),
 'right_knee': (629, 449),
 'right_ankle': (630, 517),
 'left_hip': (691, 413),
 'left_knee': (705, 448),
 'left_ankle': (708, 514),
 'right_eye': (868, 297),
 'left_eye': (876, 302),
 'right_ear': (857, 294),
 'left_ear': (687, 363)}

In [14]:
import pandas as pd

In [21]:
xx = [{k:v[0] for k,v in d.items()} for d in cc]
yy = [{k:v[1] for k,v in d.items()} for d in cc]

In [26]:
yy[0]

{'nose': 304,
 'neck': 324,
 'right_shoulder': 320,
 ' right_elbow': 415,
 'right_wrist': 354,
 'left_shoulder': 329,
 'left_elbow': 431,
 'left_wrist': 416,
 'right_hip': 413,
 'right_knee': 449,
 'right_ankle': 517,
 'left_hip': 413,
 'left_knee': 448,
 'left_ankle': 514,
 'right_eye': 297,
 'left_eye': 302,
 'right_ear': 294,
 'left_ear': 363}

In [22]:
len(xx), len(yy)

(241, 241)

In [15]:
df = pd.DataFrame(cc)

In [16]:
df.head()

,right_elbow,left_ankle,left_ear,left_elbow,left_eye,left_hip,left_knee,left_shoulder,left_wrist,neck,nose,right_ankle,right_ear,right_eye,right_hip,right_knee,right_shoulder,right_wrist
0,"(626, 415)","(708, 514)","(687, 363)","(726, 431)","(876, 302)","(691, 413)","(705, 448)","(870, 329)","(691, 416)","(854, 324)","(871, 304)","(630, 517)","(857, 294)","(868, 297)","(651, 413)","(629, 449)","(839, 320)","(391, 354)"
1,"(625, 414)","(708, 514)","(687, 361)","(726, 431)","(876, 302)","(691, 413)","(705, 448)","(871, 329)","(691, 416)","(854, 325)","(871, 304)","(630, 517)","(857, 294)","(868, 297)","(651, 413)","(629, 449)","(839, 320)","(392, 354)"
2,"(627, 415)","(708, 514)","(687, 363)","(727, 432)","(876, 302)","(691, 413)","(704, 447)","(870, 332)","(694, 417)","(853, 326)","(871, 304)","(735, 163)","(858, 294)","(868, 297)","(650, 413)","(629, 448)","(836, 319)","(391, 355)"
3,"(379, 329)","(708, 514)","(687, 364)","(727, 432)","(876, 301)","(691, 413)","(704, 447)","(871, 332)","(693, 417)","(854, 326)","(871, 304)","(735, 163)","(858, 295)","(868, 297)","(650, 413)","(629, 449)","(837, 320)","(392, 354)"
4,"(622, 416)","(708, 514)","(687, 363)","(902, 345)","(876, 302)","(690, 412)","(704, 447)","(872, 330)","(695, 417)","(854, 325)","(871, 304)","(734, 164)","(858, 294)","(868, 297)","(650, 413)","(629, 450)","(837, 321)","(392, 355)"


In [17]:
df.shape

(241, 18)

In [23]:
xx_ = pd.DataFrame(xx)
yy_ = pd.DataFrame(yy)

In [25]:
xx_.to_csv("./x_c.csv", index=False)
yy_.to_csv("./y_c.csv", index=False)

In [27]:
!pwd

/home/ashish/Downloads/Repos/keras_Realtime_Multi-Person_Pose_Estimation-master
